In [11]:
from transformers import BertTokenizer
from torch.utils.data import DataLoader, TensorDataset
from utils import pdfs_to_df, tokenize_df_of_texts
import torch

In [1]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/Users/peterargo/anaconda3/envs/myenv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
directory = './test_pdfs'
df = pdfs_to_df(directory)
df = tokenize_df_of_texts(df)

drop_cols = [col for col in df.columns if col not in ['Document', 'Text', 'Original_Text', 'Path', 'tokens']]
print(drop_cols)

df = df.drop(columns=drop_cols)

./test_pdfs/2101.00031.pdf
./test_pdfs/2101.01089.pdf
./test_pdfs/2101.00182.pdf
./test_pdfs/2101.00525.pdf
./test_pdfs/2101.01017.pdf
./test_pdfs/2101.00005.pdf
./test_pdfs/2101.00763.pdf
./test_pdfs/2101.01291.pdf
./test_pdfs/2101.00831.pdf
./test_pdfs/2101.01094.pdf
./test_pdfs/2101.00572.pdf
processing text...
making lower-case...
Removing non-text elements (extra whitespaces)...
Removing unnecessary whitespace and special characters...
Removing line breaks...
Removing gibberish...
Removing unicode...
remove single letters or super large words (so big they don't make sense)...
done cleaning.

tokenize the processed text...
['Abstract', 'Abstract_Original', 'sha_256', 'language', 'language_probability', 'Authors', 'Title', 'url', 'date', 'token_embeddings']


In [9]:
# Example data
train_texts = df['tokens'].to_list()[0:8]  # List of your tokenized texts
train_labels = df['tokens'].to_list()[8:]  # List of corresponding labels (e.g., relevance to query)

In [13]:
# Tokenize text data and create input features
input_ids = []
attention_masks = []
for text in train_texts:
    encoded_text = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt',
    )
    input_ids.append(encoded_text['input_ids'])
    attention_masks.append(encoded_text['attention_mask'])



In [ ]:
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(train_labels)

# Create a dataset and dataloader
dataset = TensorDataset(input_ids, attention_masks, labels)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
